In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors
%matplotlib inline

In [ ]:
def julia_image(N,cmap='gnuplot2'):
    z = julia_set(N)
    dpi = 72
    width = 1+ N//dpi
    height = 1+ N//dpi
    
    fig, ax = plt.subplots(figsize=(width, height),dpi=72)
    
    ax.imshow(z,cmap=cmap,origin='lower')

In [ ]:
%load_ext cython

In [ ]:
%%cython

import numpy as np
cimport numpy as np
import cython

@cython.boundscheck(False)
def julia_cython(int N):
    cdef np.ndarray[np.uint8_t, ndim=2] T = np.empty((N, 2*N), dtype=np.uint8)
    cdef double complex c = -0.835 - 0.2321j
    cdef double complex z
    cdef int J, I
    cdef double h = 2.0/N
    cdef double x, y
    for J in range(N):
        for I in range(2*N):
            y = -1.0 + J*h
            x = -2.0 + I*h
            T[J,I] = 0
            z = x + 1j * y
            while z.imag**2 + z.real**2 <= 4:
                z = z**2 + c
                T[J,I] += 1

    return T

In [ ]:
julia_set = julia_cython
julia_image(1000)

In [ ]:
%timeit julia_set(1000)

In [ ]:
%%cython

import numpy as np
cimport numpy as np
import cython
from cython.parallel cimport prange

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef julia_cython_opt(int N):
    cdef np.uint8_t[:,:] T = np.empty((N, 2*N), dtype=np.uint8)
    cdef double creal = -0.835, cimag = - 0.2321
    cdef double zreal, zimag, zreal2, zimag2
    cdef int J, I
    cdef double h = 2.0/N
    for J in range(N):
        for I in range(2*N):
            zimag = -1.0 + J*h
            zreal = -2.0 + I*h
            T[J,I] = 0
            zreal2 = zreal*zreal
            zimag2 = zimag*zimag
            while zimag2 + zreal2 <= 4:
                zimag = 2* zreal*zimag + cimag
                zreal = zreal2 - zimag2 + creal
                zreal2 = zreal*zreal
                zimag2 = zimag*zimag 
                T[J,I] += 1
                 
    return T

In [ ]:
from numba import jit

@jit
def julia_numba(N):
    T = np.empty((N, 2*N), dtype=np.uint8)
    creal = -0.835
    cimag = - 0.2321
    h = 2.0/N
    for J in range(N):
        for I in range(2*N):
            zimag = -1.0 + J*h
            zreal = -2.0 + I*h
            T[J,I] = 0
            zreal2 = zreal*zreal
            zimag2 = zimag*zimag
            while zimag2 + zreal2 <= 4:
                zimag = 2* zreal*zimag + cimag
                zreal = zreal2 - zimag2 + creal
                zreal2 = zreal*zreal
                zimag2 = zimag*zimag 
                T[J,I] += 1 
    return T

In [ ]:
from numba import jit, guvectorize, int32, complex128, complex64, uint8

@guvectorize([(complex128[:], uint8[:])], '(n)->(n)', target='parallel')
def julia_vect(Z,T):
    creal = -0.835
    cimag = - 0.2321
    for i in range(Z.shape[0]):
        zimag = Z[i].imag
        zreal = Z[i].real
        T[i] = 0
        zreal2 = zreal*zreal
        zimag2 = zimag*zimag
        while zimag2 + zreal2 <= 4:
            zimag = 2* zreal*zimag + cimag
            zreal = zreal2 - zimag2 + creal
            zreal2 = zreal*zreal
            zimag2 = zimag*zimag 
            T[i] += 1    
            

def julia_numba_vect(N):
    r1 = np.linspace(-2.0, 2.0, 2*N)
    r2 = np.linspace(-1.0, 1.0, N, dtype=np.float32)
    Z = r1 + r2[:,None]*1j
    T = julia_vect(Z)
    return T

In [ ]:
from numba import jit, guvectorize, int32, complex128, complex64, uint8

@guvectorize([(complex128[:], uint8[:])], '(n)->(n)', target='cuda')
def julia_cuda(Z,T):
    creal = -0.835
    cimag = - 0.2321
    for i in range(Z.shape[0]):
        zimag = Z[i].imag
        zreal = Z[i].real
        T[i] = 0
        while True:
            zreal2 = zreal*zreal
            zimag2 = zimag*zimag
            if zimag2 + zreal2 > 4:
                break;
            zimag = 2* zreal*zimag + cimag
            zreal = zreal2 - zimag2 + creal
            T[i] += 1 

def julia_numba_cuda(N):
    r1 = np.linspace(-2.0, 2.0, 2*N)
    r2 = np.linspace(-1.0, 1.0, N, dtype=np.float32)
    Z = r1 + r2[:,None]*1j
    T = julia_cuda(Z)
    return T